<a href="https://colab.research.google.com/github/lawrennd/qig-code/blob/main/examples/origin_paper_simulation_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Origin paper: simulation experiments

### Neil D. Lawrence

### December 2025

This notebook accompanies *The Origin of the Inaccessible Game* (`the-inaccessible-game-origin.tex`). It collects simulation experiments that highlight the paper’s main structural points, using the QIG implementation (especially `qig/exponential_family.py`, `qig/dynamics.py`, `qig/generic.py`).

In [ ]:
# Auto-install QIG package if not available
import os

try:
    import qig
except ImportError:
    print("📦 Installing QIG package...")
    %pip install -q git+https://github.com/lawrennd/qig-code.git
    import qig
    print("✓ QIG package installed!")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, suppress=True)

from qig.core import marginal_entropies, von_neumann_entropy
from qig.pair_operators import bell_state_density_matrix

from qig.exponential_family import QuantumExponentialFamily
from qig.dynamics import InaccessibleGameDynamics, GenericDynamics


In [ ]:
# Plot configuration (match examples/boring_game_dynamics.ipynb)
plt.style.use('seaborn-v0_8-whitegrid' if 'seaborn-v0_8-whitegrid' in plt.style.available else 'default')
big_wide_figsize = (10, 5)
big_figsize = (8, 6)
plt.rcParams.update({
    'font.size': 14,
    'font.family': 'serif',
    'axes.labelsize': 18,
    'axes.titlesize': 16,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 12,
})

os.makedirs('./diagrams', exist_ok=True)
print("✓ Configuration complete")


## Notes on runtime

- Fastest configuration: `n_pairs=1, d=2` (one qubit pair).
- Paper-level examples typically use qutrits: `d=3`.
- Some experiments compute Jacobians / third cumulants; those are heavier.

## Experiment 1 — The classical origin paradox vs the quantum resolution

**Paper**: Section *The Classical Conflict at the Origin* and *Resolution via von Neumann Entropy*.

**Goal**: Show that the “origin” configuration is impossible classically but allowed quantumly via negative conditional entropy (entanglement).

In [ ]:
def shannon_entropy(p):
    p = np.asarray(p, dtype=float)
    p = p[p > 1e-15]
    return float(-np.sum(p * np.log(p)))


def shannon_entropies_from_joint(Pxy):
    Pxy = np.asarray(Pxy, dtype=float)
    Pxy = Pxy / Pxy.sum()
    Px = Pxy.sum(axis=1)
    Py = Pxy.sum(axis=0)
    Hx = shannon_entropy(Px)
    Hy = shannon_entropy(Py)
    Hxy = shannon_entropy(Pxy.flatten())
    Hx_given_y = Hxy - Hy
    return Hx, Hy, Hxy, Hx_given_y


# Classical attempt: make H(X,Y)=0 (a point mass) while H(X),H(Y)>0.
# Any point mass forces both marginals to be point masses -> H(X)=H(Y)=0.
P_point = np.array([[0, 0], [0, 1]], dtype=float)
Hx, Hy, Hxy, Hx_given_y = shannon_entropies_from_joint(P_point)
print("Classical point-mass joint:")
print("H(X)=", Hx, "H(Y)=", Hy, "H(X,Y)=", Hxy, "H(X|Y)=", Hx_given_y)


# Quantum Bell state: H(AB)=0 but H(A)=H(B)=log d.
for d in [2, 3]:
    rho_AB = bell_state_density_matrix(d)
    hA, hB = marginal_entropies(rho_AB, dims=[d, d])
    H_AB = von_neumann_entropy(rho_AB)
    H_A_given_B = H_AB - hB

    print(f"\nQuantum Bell (d={d}):")
    print("H(AB)=", H_AB)
    print("H(A)=", hA, "H(B)=", hB)
    print("H(A|B)=", H_A_given_B, "(negative)")


## Experiment 2 — Approaching the pure-state boundary: ‖θ‖ diverges

**Paper**: Subsection *Pure States and the Quantum Exponential Family*.

**Goal**: Show that as we regularize the Bell state with smaller ε, the represented state approaches the boundary and the natural parameter norm grows rapidly.

In [ ]:
exp = QuantumExponentialFamily(n_pairs=1, d=2, pair_basis=True)

log_eps_list = np.linspace(-2, -20, 10)  # eps = exp(log_eps)
rows = []

for log_eps in log_eps_list:
    theta = exp.get_bell_state_parameters(log_epsilon=float(log_eps))
    rho = exp.rho_from_theta(theta)

    eigvals = np.linalg.eigvalsh(rho)
    min_eig = float(np.min(eigvals.real))

    H = exp.von_neumann_entropy(theta)
    C, a = exp.marginal_entropy_constraint(theta)
    G = exp.fisher_information(theta)

    # In the paper (and qig.dynamics.flow), affine entropy production is:
    #   dH/dτ = θ^T G Π G θ
    a_norm_sq = float(a @ a)
    Pi = np.eye(len(theta)) if a_norm_sq < 1e-12 else (np.eye(len(theta)) - np.outer(a, a) / a_norm_sq)
    dH_dtau = float(theta @ G @ Pi @ G @ theta)

    rows.append((log_eps, float(np.linalg.norm(theta)), min_eig, float(H), dH_dtau))

print("log_eps    ||theta||     min_eig(rho)     H(rho)       dH/dtau")
for log_eps, th_norm, min_eig, H, dH_dtau in rows:
    print(f"{log_eps:7.1f}  {th_norm:11.3e}  {min_eig:12.3e}  {H:10.3e}  {dH_dtau:10.3e}")


## Experiment 3 — Entropy-time integration: H increases linearly by construction

**Paper**: Subsection *Pure States and the Quantum Exponential Family* (entropy-time definition) and Section *Quantum Exponential Family and Constrained Flow*.

**Goal**: Integrate with `entropy_time=True` and verify that the joint entropy increases by ~`dt` per step (up to numerical effects).

In [ ]:
exp = QuantumExponentialFamily(n_pairs=1, d=2, pair_basis=True)

theta0 = exp.get_bell_state_parameters(epsilon=1e-3)

dyn = InaccessibleGameDynamics(exp)
res = dyn.solve(theta0, n_steps=200, dt=0.01, entropy_time=True, project=True, project_every=5, verbose=False)
traj = res["trajectory"]

H = np.array([exp.von_neumann_entropy(th) for th in traj])

dH = np.diff(H)
print("First 10 entropy increments (should be near dt=0.01):")
print(np.round(dH[:10], 6))
print("Total dH vs expected:", float(H[-1] - H[0]), "vs", 200 * 0.01)

plt.figure(figsize=big_wide_figsize)
plt.plot(H, lw=2)
plt.xlabel("step")
plt.ylabel("joint entropy H(ρ)")
plt.title("Entropy-time integration: H increases ~linearly")
plt.tight_layout()
plt.show()


## Experiment 4 — Constraint drift without projection (why `solve()` exists)

**Paper**: Section *Quantum Exponential Family and Constrained Flow* (constraint enforcement) and the numerical theme: constraint preservation is nontrivial.

**Goal**: Compare the deprecated `integrate()` (no projection) to `solve()` (Newton projection).

In [ ]:
rng = np.random.default_rng(0)

exp = QuantumExponentialFamily(n_pairs=1, d=2, pair_basis=True)
theta0 = 0.2 * rng.normal(size=exp.n_params)

dyn = InaccessibleGameDynamics(exp)

# (A) Unstable ODE integration (constraint drift)
res_ode = dyn.integrate(theta0, t_span=(0.0, 0.5), n_points=60)
C_ode = res_ode["constraint"]

# (B) Stable projected solver
res_proj = dyn.solve(theta0, n_steps=60, dt=0.01, entropy_time=False, project=True, project_every=1, verbose=False)
traj = res_proj["trajectory"]
C_proj = np.array([
    float(np.sum(marginal_entropies(exp.rho_from_theta(th), exp.dims)))
    for th in traj
])

print("ODE drift: max|C(t)-C(0)| =", float(np.max(np.abs(C_ode - C_ode[0]))))
print("Projected: max|C(t)-C(0)| =", float(np.max(np.abs(C_proj - C_proj[0]))))

plt.figure(figsize=big_wide_figsize)
plt.plot(C_ode - C_ode[0], label="integrate() drift")
plt.plot(C_proj - C_proj[0], label="solve() projected")
plt.axhline(0.0, color="k", lw=1)
plt.xlabel("step (or eval index)")
plt.ylabel("C(t) - C(0)")
plt.title("Constraint drift without projection")
plt.legend()
plt.tight_layout()
plt.show()


## Experiment 5 — “Boring” dynamics from the LME/Bell origin

**Paper**: Section *The Classical Conflict at the Origin* (origin as LME) and Section *Quantum Exponential Family and Constrained Flow*.

**Goal**: Start from a (regularized) Bell origin and show:
- constraint sum $C=\sum_i h_i$ stays constant,
- joint entropy increases,
- mutual information decreases.

This matches the “boring game” narrative in `examples/boring_game_dynamics.ipynb`.

In [ ]:
exp = QuantumExponentialFamily(n_pairs=1, d=3, pair_basis=True)

theta0 = exp.get_bell_state_parameters(epsilon=1e-3)

dyn = InaccessibleGameDynamics(exp)
res = dyn.solve(theta0, n_steps=200, dt=0.02, entropy_time=True, project=True, project_every=5, verbose=False)
traj = res["trajectory"]

H = []
C = []
I = []
for th in traj:
    rho = exp.rho_from_theta(th)
    h = marginal_entropies(rho, exp.dims)
    H_val = float(von_neumann_entropy(rho))
    C_val = float(np.sum(h))
    H.append(H_val)
    C.append(C_val)
    I.append(C_val - H_val)

H = np.array(H)
C = np.array(C)
I = np.array(I)

print("C drift (should be tiny):", float(np.max(np.abs(C - C[0]))))
print("H start/end:", float(H[0]), "->", float(H[-1]))
print("I start/end:", float(I[0]), "->", float(I[-1]))

plt.figure(figsize=big_wide_figsize)
plt.plot(H, label="H(ρ)")
plt.plot(I, label="I = C - H")
plt.plot(C, label="C = Σ h_i", linestyle="--")
plt.xlabel("step")
plt.title("Boring game from Bell origin: C fixed, H↑, I↓")
plt.legend()
plt.tight_layout()
plt.show()


## Experiment 6 — Constraint engages away from LME: monitor ‖a(θ)‖

**Paper**: Section *Quantum Exponential Family and Constrained Flow* and Subsection *Constraint Linearisation at the Origin*.

**Goal**: Start from a generic interior point and monitor the constraint-gradient norm $\|a(\theta)\|$ to see when the constraint is “active”.

In [ ]:
rng = np.random.default_rng(1)
exp = QuantumExponentialFamily(n_pairs=1, d=2, pair_basis=True)

th0 = 0.6 * rng.normal(size=exp.n_params)

C0, a0 = exp.marginal_entropy_constraint(th0)
print("Initial C=", C0)
print("Initial ||a||=", float(np.linalg.norm(a0)))

# Run constrained dynamics

dyn = InaccessibleGameDynamics(exp)
res = dyn.solve(th0, n_steps=300, dt=0.01, entropy_time=True, project=True, project_every=1, verbose=False)
traj = res["trajectory"]

a_norms = np.array([float(np.linalg.norm(exp.marginal_entropy_constraint(th)[1])) for th in traj])

plt.figure(figsize=big_wide_figsize)
plt.plot(a_norms)
plt.xlabel("step")
plt.ylabel("||a(θ)||")
plt.title("Constraint engagement proxy: ||∇C|| along trajectory")
plt.tight_layout()
plt.show()


## Experiment 7 — Structural identity: local basis vs pair basis

**Paper**: Subsection *Constraint Linearisation at the Origin* and Subsection *GENERIC-like decomposition: linearised structure*.

**Goal**: Compare the identity hinted in the code comments:
- local basis: (approximately) \(G(\theta)\,\theta \approx -a(\theta)\)
- pair basis: the identity is typically broken.

Here \(a(\theta)=\nabla C(\theta)\) with \(C(\theta)=\sum_i h_i\).

In [ ]:
rng = np.random.default_rng(2)

def structural_gap(exp_fam, theta):
    G = exp_fam.fisher_information(theta)
    _, a = exp_fam.marginal_entropy_constraint(theta)
    gap = G @ theta + a
    return float(np.linalg.norm(gap)), float(np.linalg.norm(G @ theta)), float(np.linalg.norm(a))

# Local basis: separable-only chart
exp_local = QuantumExponentialFamily(n_sites=2, d=2, pair_basis=False)
th_local = 0.2 * rng.normal(size=exp_local.n_params)

# Pair basis: entanglement-capable chart
exp_pair = QuantumExponentialFamily(n_pairs=1, d=2, pair_basis=True)
th_pair = 0.2 * rng.normal(size=exp_pair.n_params)

print("Local basis  ||Gθ + a||, ||Gθ||, ||a||:")
print(structural_gap(exp_local, th_local))

print("Pair basis   ||Gθ + a||, ||Gθ||, ||a||:")
print(structural_gap(exp_pair, th_pair))


## Experiment 8 — GENERIC decomposition: extract H_eff and compare diffusion vs Milburn

**Paper**: Subsection *GENERIC-like decomposition: linearised structure* and Section *Connection to intrinsic decoherence and steepest entropy ascent*.

**Goal**:
1. compute `M`, split into `S` and `A`,
2. extract the effective Hamiltonian `H_eff` from `A`,
3. compare the Kubo–Mori diffusion operator to the Milburn approximation near equilibrium.

In [ ]:
from qig.generic import compare_diffusion_methods

exp = QuantumExponentialFamily(n_pairs=1, d=2, pair_basis=True)

rng = np.random.default_rng(3)
th0 = 0.1 * rng.normal(size=exp.n_params)

gdyn = GenericDynamics(exp, method='duhamel')
res = gdyn.integrate_with_monitoring(th0, t_span=(0.0, 0.2), n_points=15, compute_diffusion=False)
traj = res['theta']

th = traj[-1]
info = gdyn.compute_generic_decomposition(th)

S = info['S']
H_eff = info['H_eff']

report = compare_diffusion_methods(S, th, H_eff, exp, gamma=1.0, tol=5e-2)
report.print_summary(verbose=True)


## Experiment 9 — Qutrit optimality from an additive “level budget”

**Paper**: Section *Qutrit Optimality and Origin Structure*.

**Goal**: Under an additive budget model, maximise
$$
\frac{\log d}{d}
$$
over integers \(d\ge 2\). The optimizer is \(d=3\).

We also sanity-check that the Bell-origin mutual information is \(2\log d\) (for one pair).

In [ ]:
def score(d):
    return np.log(d) / d

ds = np.arange(2, 21)
scores = np.array([score(int(d)) for d in ds])

best_d = int(ds[np.argmax(scores)])
print("d values:", ds)
print("(log d)/d:", np.round(scores, 6))
print("best d:", best_d)

# Sanity-check against QIG mutual information at the (regularized) Bell origin
for d in [2, 3, 4]:
    exp = QuantumExponentialFamily(n_pairs=1, d=int(d), pair_basis=True)
    th = exp.get_bell_state_parameters(epsilon=1e-6)
    I = exp.mutual_information(th)
    print(f"d={d}: I(theta)≈{I:.6f}  vs  2 log d={2*np.log(d):.6f}")
